In [ ]:
# code_15_BERT_PROPN.py
# !pip install transformers
import os

print(os.getcwd())
# GAP dataset: https://github.com/google-research-datasets/gap-coreference

In [2]:

#提取代词特征

import pandas as pd
import pickle
import torch
from tqdm import tqdm
from transformers import BertTokenizer, BertModel, BertConfig




/home/xhm/.conda/envs/dgl/lib/python3.6/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# BERT_PATH = '/home/xhm/nlp/pytorch-GNN/pytorch-GNN-2nd/huggingface/bert-base-uncased'

# tokenizer = BertTokenizer.from_pretrained(BERT_PATH)

# print(tokenizer.tokenize('I have a good time, thank you.'))

# bert = BertModel.from_pretrained(BERT_PATH)

# print('load bert model over')


In [4]:

#指定设备
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

#读取数据    
# 变量命名有问题？
df_test = pd.read_csv("../gap-coreference/gap-development.tsv", delimiter="\t")
df_train_val = pd.concat([
    pd.read_csv("../gap-coreference/gap-test.tsv", delimiter="\t"),
    pd.read_csv("../gap-coreference/gap-validation.tsv", delimiter="\t")
], axis=0)



cuda:0


In [5]:

def getmodel():
    #加载词表文件tokenizer
    tokenizer = BertTokenizer.from_pretrained('../huggingface/bert-base-uncased')

    #添加特殊词
    special_tokens_dict = {'additional_special_tokens': ["[THISISA]", "[THISISB]", "[THISISP]"]}
    tokenizer.add_special_tokens(special_tokens_dict)  #添加特殊词
    print('from getmodel', tokenizer.additional_special_tokens, tokenizer.additional_special_tokens_ids)

    model = BertModel.from_pretrained('../huggingface/bert-base-uncased')  #加载模型
    return tokenizer, model


############################


def insert_tag(row, hasbrack=True):
    # P: 目标代词   A: 候选名称1    B: 候选名称2
    orgtag = [" [THISISA] ", " [THISISB] ", " [THISISP] "]
    if hasbrack == False:
        orgtag = [" THISISA ", " THISISB ", " THISISP "]

    #按照插入的位置，从大到小排序[(383, ' THISISP '), (366, ' THISISB '), (352, ' THISISA ')]
    to_be_inserted = sorted([
        (row["A-offset"], orgtag[0]),
        (row["B-offset"], orgtag[1]),
        (row["Pronoun-offset"], orgtag[2])], key=lambda x: x[0], reverse=True)

    text = row["Text"]  #len 443
    for offset, tag in to_be_inserted:  #先插最后的，不会影响前面
        text = text[:offset] + tag + text[offset:]  #（插到每个代词的前面）
    return text  #len 470 (443+3*9)


# 将标签分离，并返回标签偏移位置
def tokenize(sequence_ind, tokenizer, sequence_mask=None):  #将标签分离，并返回标签偏移位置
    entries = {}
    final_tokens = []
    final_mask = []

    for i, one in enumerate(sequence_ind):
        if one in tokenizer.additional_special_tokens_ids:
            tokenstr = tokenizer.convert_ids_to_tokens(one)
            entries[tokenstr] = len(final_tokens)
            continue
        final_tokens.append(one)
        if sequence_mask is not None:
            final_mask.append(sequence_mask[i])
    return final_tokens, (entries["[THISISA]"], entries["[THISISB]"], entries["[THISISP]"]), final_mask


def savepkl(df, name):
    bert_prediction = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        #循环内部
        text = insert_tag(row)  #插入标签
        sequence_ind = tokenizer.encode(text)  #向量化
        tokens, offsets, _ = tokenize(sequence_ind, tokenizer)  #获取标签偏移
        token_tensor = torch.LongTensor([tokens]).to(device)
        #         bert_outputs,bert_last_outputs=  model(token_tensor)  #[1, 107, 768] , [1, 768]
        #         print(type(model))
        #         <class 'str'> last_hidden_state
        #         print(type(bert_outputs), bert_outputs)
        #         print(type(bert_last_outputs), bert_last_outputs)
        bert_outputs, bert_last_outputs = model(token_tensor)[:2]  #[1, 107, 768] , [1, 768]
        #         print(offsets)
        extracted_outputs = bert_outputs[:, offsets, :]  #根据偏移位置抽取特征向量
        bert_prediction.append(extracted_outputs.cpu().numpy())
    pickle.dump(bert_prediction, open(name, "wb"))



In [6]:

if __name__ == '__main__':
    tokenizer, model = getmodel()
    model.to(device)
    torch.set_grad_enabled(False)

    savepkl(df_test, 'output/test_bert_outputs_forPROPN.pkl')
    savepkl(df_train_val, 'output/bert_outputs_forPROPN.pkl')

from getmodel ['[THISISA]', '[THISISB]', '[THISISP]'] [30522, 30523, 30524]


Some weights of the model checkpoint at ../huggingface/bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2454/2454 [00:24<00:00, 101.80it/s]
